In [1]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import xception

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

Using TensorFlow backend.


In [2]:
!git clone https://github.com/deepanrajm/Covid-19.git

Cloning into 'Covid-19'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 62 (delta 0), reused 59 (delta 0), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [0]:
!ls

In [0]:
INIT_LR = 1e-3
EPOCHS = 25
# Load the training data set by looping over every image file
for image_file in Path("Covid-19/Classifying_Covid_19/training_dataset").glob("**/*.jpeg"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "normal" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [4]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = xception.preprocess_input(x_train)

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = xception.Xception(weights='imagenet', include_top=False, input_shape=(73, 73, 3))

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")


83689472/83683744 [==============================] - 9s 0us/step


['y_train.dat']

In [0]:
# Load data set of extracted features
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam

In [0]:
# Create a model and add layers
model = Sequential()

# Add layers to our model
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)


In [0]:
# Train the model
model.fit(
    x_train,
    y_train,
    epochs=EPOCHS,
    shuffle=True,
    verbose=1
)

# Save the trained model to a file so we can use it to make predictions later
model.save("Covid_feature_classifier_model.h5")

In [0]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [16]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("Covid-19/Classifying_Covid_19/test_dataset").glob("**/*.jpeg"):
    print (image_file)
    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)


# Convert the list of test images to a numpy array
    x_test = np.array(image_array)

    x_test = np.expand_dims(x_test, axis=0)

# Normalize test data set to 0-to-1 range
    x_test = xception.preprocess_input(x_test)

    # Load our trained classifier model
    model = load_model("Covid_feature_classifier_model.h5")

    # Extract features for each image (all in one pass)
    features_x = feature_extractor.predict(x_test)

    # Given the extracted features, make a final prediction using our own model
    predictions = model.predict(features_x)

    print (predictions)

Covid-19/Classifying_Covid_19/test_dataset/Normal.jpeg
[[1.]]
Covid-19/Classifying_Covid_19/test_dataset/Covid.jpeg
[[1.]]
